In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Function to read data from a CSV file
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df

# Function to convert date format from MM/DD/YYYY to datetime object
def convert_to_datetime(date_str):
    try:
        return datetime.strptime(date_str, '%m/%d/%Y')
    except ValueError:
        return None

# Function to filter data from the last 5 days and future dates
def filter_recent_and_future_data(df, days=5):
    today = datetime.today()
    start_date = today - timedelta(days=days)
    df['Date'] = df['Date'].apply(convert_to_datetime)
    recent_and_future_df = df[(df['Date'] >= start_date)]
    return recent_and_future_df

# File paths
input_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/Combined_Split_Data.csv'
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/TestData.csv'

# Read data from the CSV file
data_df = read_csv(input_file_path)

# Filter data from the last 5 days and future dates
recent_and_future_data_df = filter_recent_and_future_data(data_df)

# Convert the Date column back to MM/DD/YYYY format
recent_and_future_data_df['Date'] = recent_and_future_data_df['Date'].dt.strftime('%m/%d/%Y')

# Save the recent and future data to a new CSV file
recent_and_future_data_df.to_csv(output_file_path, index=False)

print(f"Recent and future data has been saved to {output_file_path}")


Recent and future data has been saved to /Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/TestData.csv


/var/folders/r6/0smcfgt9415gwpc747f7ghth0000gn/T/ipykernel_5774/1657898269.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_and_future_data_df['Date'] = recent_and_future_data_df['Date'].dt.strftime('%m/%d/%Y')


In [2]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to perform Google search and get links
def get_search_results(query, num_results=2):
    search_results = search(query, num_results=num_results)
    return list(search_results)

# Function to extract content from a search result
def extract_content_from_url(url, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract all text content from the page
            return soup.get_text(separator="\n")
        else:
            return f"Failed to retrieve webpage. Status code: {response.status_code}"
    except requests.exceptions.Timeout:
        return "Request timed out"
    except requests.exceptions.RequestException as e:
        return f"Request failed: {e}"

# Function to search for specific terms in the content
def search_for_terms(content, terms):
    for term in terms:
        if term.lower() in content.lower():
            return True
    return False

# Read symbols and dates from the CSV file
input_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/TestData.csv'
df_symbols = pd.read_csv(input_file_path)
symbols = df_symbols['Symbol'].tolist()
dates = df_symbols['Date'].tolist()

# Terms to search for very basic
terms = [" round ", "rounded", "rounding"]

# Prepare the result list
results = []

# Process each symbol and its corresponding date
for symbol, date in zip(symbols, dates):
    query = f"{symbol} Reverse split press release"
    print(f"\nSearching for: {query}")

    # Get the top 2 search results
    links = get_search_results(query, num_results=1)
    
    # Flag to track if "DO BUY" should be set
    do_buy_flag = False

    # Extract and check content from the search results
    for i, link in enumerate(links):
        print(f"\nExtracting content from search result {i+1}: {link}")
        content = extract_content_from_url(link)
        
        # Check for specific terms in the content
        if search_for_terms(content, terms):
            do_buy_flag = True
            break

    # Determine final decision
    decision = "ROUNDING" if do_buy_flag else "FORGO"
    results.append([date, symbol, decision])
    print(f"Decision for {symbol}: {decision}")

# Create a DataFrame from the results
result_df = pd.DataFrame(results, columns=['Date', 'Symbol', 'Decision'])

# Save the results to a new CSV file
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/Results.csv'
result_df.to_csv(output_file_path, index=False)

print(f"\nResults have been saved to {output_file_path}")


/Users/ardadinc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(



Searching for: AMWL Reverse split press release

Extracting content from search result 1: https://investors.amwell.com/news/news-details/2024/Amwell-Announces-Reverse-Stock-Split/default.aspx

Extracting content from search result 2: https://markets.businessinsider.com/news/stocks/amwell-announces-reverse-stock-split-1033514553

Extracting content from search result 3: https://www.investing.com/news/company-news/amwell-announces-reverse-stock-split-to-meet-nyse-requirements-93CH-3501729
Decision for AMWL: FORGO

Searching for: FRES Reverse split press release

Extracting content from search result 1: https://www.starequity.com/news-releases/news-release-details/star-equity-holdings-announces-reverse-stock-split

Extracting content from search result 2: https://ir.korewireless.com/news-events/press-releases/detail/230/kore-to-effect-a-reverse-stock-split
Decision for FRES: ROUNDING

Searching for: EFSH Reverse split press release

Extracting content from search result 1: https://www.na

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



Extracting content from search result 3: https://nasdaqtrader.com/TraderNews.aspx?id=eca2023-150
Decision for ASLN: FORGO

Searching for: NUTX Reverse split press release

Extracting content from search result 1: https://www.prnewswire.com/news-releases/nutex-health-announces-1-for-10-reverse-stock-split-302184974.html
Decision for NUTX: ROUNDING

Searching for: ASST Reverse split press release

Extracting content from search result 1: https://investors.assetentities.com/news-releases/news-release-details/asset-entities-inc-announces-reverse-stock-split-regain
Decision for ASST: ROUNDING

Searching for: BURU Reverse split press release

Extracting content from search result 1: https://ir.nuburu.net/news/news-details/2024/NUBURU-Announces-Revised-Date-for-Strategic-1-for-40-Reverse-Stock-Split/default.aspx

Extracting content from search result 2: https://finance.yahoo.com/news/nuburu-announces-revised-date-strategic-234400570.html

Extracting content from search result 3: https://seek

<h2>-----------------------------Add Price Data to RevSplits-----------------------------<h2>

In [3]:
import pandas as pd
import yfinance as yf

# Path to the input CSV file (X.csv)
input_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/Results.csv'
# Path to the output CSV file (Y.csv)
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/Results.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path)

# Function to get the latest stock price from yfinance and round to 2 decimals
def get_latest_stock_price(symbol):
    try:
        stock = yf.Ticker(symbol)
        hist = stock.history(period="1d")
        if not hist.empty:
            return round(hist['Close'].iloc[0], 3)
        else:
            return None
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Add a new column for the stock price
df['Price'] = df['Symbol'].apply(get_latest_stock_price)

# Save the updated DataFrame to the output CSV file
df.to_csv(output_file_path, index=False)

print(f"Updated data with stock prices has been saved to {output_file_path}")


$ALTAF: possibly delisted; No price data found  (period=1d)


$ALTAF: possibly delisted; No price data found  (period=1d)


$MYBUF: possibly delisted; No price data found  (period=1d)
$SPEYF: possibly delisted; No price data found  (period=1d)


$MYBUF: possibly delisted; No price data found  (period=1d)
$SPEYF: possibly delisted; No price data found  (period=1d)


$ESHSF: possibly delisted; No price data found  (period=1d)


$ESHSF: possibly delisted; No price data found  (period=1d)
Updated data with stock prices has been saved to /Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/Results.csv
